In [2]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import PyPDFLoader


from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableParallel, RunnableLambda

from langchain_core.output_parsers import StrOutputParser

from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor


In [3]:
%pip install python-dotenv
import dotenv
%load_ext dotenv
%dotenv

Note: you may need to restart the kernel to use updated packages.
The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [5]:
import os

In [4]:
questions = [
    "Qual é a visão de Euclides da Cunha sobre o ambiente natural do sertão nordestino e como ele influencia a vida dos habitantes?",
    "Quais são as principais características da população sertaneja descritas por Euclides da Cunha? Como ele relaciona essas características com o ambiente em que vivem?",
    "Qual foi o contexto histórico e político que levou à Guerra de Canudos, segundo Euclides da Cunha?",
    "Como Euclides da Cunha descreve a figura de Antônio Conselheiro e seu papel na Guerra de Canudos?",
    "Quais são os principais aspectos da crítica social e política presentes em \"Os Sertões\"? Como esses aspectos refletem a visão do autor sobre o Brasil da época?",
]

In [6]:
#Carregar modelos OPENAI - embedding e chat
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
llm = ChatOpenAI(model_name= "gpt-3.5-turbo", max_tokens = 300)

In [7]:
# Carregar PDF

pdf_path = os.path.join("..", "data", "os-sertoes.pdf")
loader = PyPDFLoader(pdf_path, extract_images=False)
pages = loader.load_and_split()


In [8]:
# Separar em chunks
text_splitter = RecursiveCharacterTextSplitter(
  chunk_size=600, 
  chunk_overlap=100, 
  length_function=len, 
  add_start_index=True,
  separators=["\n\n", "\n", ".", "!", "?", ",", " "]
)
chunks = text_splitter.split_documents(pages)

In [9]:
# Salvar chunks no ChromaDB
vectordb = Chroma(embedding_function=embeddings)

/var/folders/pp/byh0kn390x12x8dgh89r2d8r0000gn/T/ipykernel_51423/866894675.py:2: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectordb = Chroma(embedding_function=embeddings)


In [10]:
# Carregar chunks no ChromaDB
vectordb.add_documents(chunks)
naive_retriever = vectordb.as_retriever(search_kwargs={"k": 10})

In [11]:
compressor = LLMChainExtractor.from_llm(llm)

In [12]:
retriever = ContextualCompressionRetriever(
    base_retriever=naive_retriever,
    base_compressor=compressor
)

In [13]:
TEMPLATE = """
Você é um especialista em literatura brasileira. Responda a pergunta abaixo utilizando o contexto informado

Contexto: {context}
    
Pergunta: {question}
"""

rag_prompt = ChatPromptTemplate.from_template(TEMPLATE)

In [14]:
setup_retrieval = RunnableParallel({"question": RunnablePassthrough(), "context": RunnableLambda(lambda x: retriever.invoke(x["question"]))})

output_parser = StrOutputParser()

compressor_retrieval_chain = (
    setup_retrieval
    | rag_prompt
    | llm
    | output_parser
)

In [19]:
def answer_question(question: str):
    return compressor_retrieval_chain.invoke({"question": question})

In [20]:
for index, question in enumerate(questions):
    resposta = answer_question(question)
    print({"numero": index, "pergunta": question, "resposta": resposta})

{'numero': 0, 'pergunta': 'Qual é a visão de Euclides da Cunha sobre o ambiente natural do sertão nordestino e como ele influencia a vida dos habitantes?', 'resposta': 'Euclides da Cunha descreve o ambiente natural do sertão nordestino como uma região de contrastes, onde as intermitências de cheias e estiagens são comparadas às batidas do coração da maior artéria da terra. Ele também destaca a fertilidade inicial do sertão, descrevendo-o como um vale vasto e sem dono, porém alerta para a dura realidade das secas que trazem dias torturantes, atmosfera asfixiante e seca assombrosa. Essas condições extremas influenciam diretamente a vida dos habitantes do sertão, tornando-a difícil e desafiadora, marcada pela luta pela sobrevivência em um ambiente hostil e implacável.'}
{'numero': 1, 'pergunta': 'Quais são as principais características da população sertaneja descritas por Euclides da Cunha? Como ele relaciona essas características com o ambiente em que vivem?', 'resposta': 'Euclides da Cu